In [1]:
import Metashape
import numpy as np

In [2]:
demo_ms = '/home/crest/z/hwang_Pro/data/2023_hokkaido_potato/projects.psx/test_1R_Group0.psx'

In [3]:
doc = Metashape.Document()
doc.open(demo_ms)

LoadProject: path = /home/crest/z/hwang_Pro/data/2023_hokkaido_potato/projects.psx/test_1R_Group0.psx
loaded project in 0.003128 sec


Document.open(): The document is opened in read-only mode because it is already in use.


In [4]:
chunk = doc.chunk

In [9]:
marker_xyz_dict = {}
total_points = []
for m in chunk.markers:
    if m.label in ['51', '52', '54']:
        marker_xyz_dict[m.label] = m.position
        total_points.append(m.position)

marker_xyz_dict

{'51': Vector([1.3957983868862447, -0.6077544841731778, -4.323084803424499]),
 '52': Vector([1.2302009794025937, -0.5915432884245656, -4.046297350523651]),
 '54': Vector([1.6608510036856543, -0.6181216692615654, -4.15107212651686])}

In [13]:
marker_xyz = np.asarray(total_points).astype(np.float64)
marker_xyz

array([[ 1.39579839, -0.60775448, -4.3230848 ],
       [ 1.23020098, -0.59154329, -4.04629735],
       [ 1.660851  , -0.61812167, -4.15107213]])

Not use '53' because its position changed since 2R1-10

In [8]:
def points2plane(points):
    # ref: https://math.stackexchange.com/questions/99299/best-fitting-plane-given-a-set-of-points
    A = np.matrix(np.insert(points[:,0:2], obj=2, values=1, axis=1))
    b = np.matrix(points[:,2]).T
  
    fit = (A.T * A).I * A.T * b
    #errors = b - A * fit
    #residual = np.linalg.norm(errors)
  
    return float(fit[0]), float(fit[1]), -1.0, float(fit[2])  # Ax+By+Cz+D=0

def point_on_plane(points, plane_param):
    # ref: https://blog.csdn.net/soaryy/article/details/82884691
    # Plane_params = (A, B, C ,D) -> Ax + By + Cz + D =0
    A, B, C, D = plane_param
    demomin = A**2 + B**2 + C**2
    x0, y0, z0 = points[:,0], points[:,1], points[:,2]
  
    xp = ((B**2+C**2)*x0 - A*(B*y0+C*z0+D)) / demomin
    yp = ((A**2+C**2)*y0 - B*(A*x0+C*z0+D)) / demomin
    zp = ((A**2+B**2)*z0 - C*(A*x0+B*y0+D)) / demomin
  
    return np.vstack([xp, yp, zp]).T


In [14]:
params = points2plane(marker_xyz)
print(f"{params[0]}x+{params[1]}y+{params[2]}z+{params[3]}=0")

2.193003208750035x+39.47537916368634y+-1.0z+16.60726355653857=0


/tmp/ipykernel_1270400/180089087.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(fit[0]), float(fit[1]), -1.0, float(fit[2])  # Ax+By+Cz+D=0


In [15]:
# project plots boundary to point plane
plane_points = point_on_plane(marker_xyz, params)
plane_points

array([[ 1.39579839, -0.60775448, -4.3230848 ],
       [ 1.23020098, -0.59154329, -4.04629735],
       [ 1.660851  , -0.61812167, -4.15107213]])

In [24]:
def vector_mod(Ax, By, Cz):
    return np.sqrt(np.sum(np.square([Ax, By, Cz])))

def find_unit_orthogonal_set(gcp3, params):
    v1 = marker_xyz_dict['51'] - marker_xyz_dict['52']
    v3 = np.asarray(params[0:3])
    uv1 = v1 / vector_mod(*v1)
    uv3 = v3 / vector_mod(*v3)
    uv2 = np.cross(uv1, uv3)
  
    # 这边要看z向量的方向是否一致，修改uv3前面的正负号
    return np.vstack([uv1, uv2, uv3]).T   # [v1, v2, v3] -> x, y, z coordinate

def trans_coord(cvtmat, points):
    return np.linalg.inv(cvtmat).dot(points.T).T


In [25]:
cvtmat = find_unit_orthogonal_set(plane_points, params)
cvtmat

array([[ 0.5127651 ,  0.85673636,  0.05545043],
       [-0.05019726, -0.03455901,  0.99814123],
       [-0.8570602 ,  0.51459545, -0.02528516]])

In [26]:
plot_trans = trans_coord(cvtmat, plane_points)
plot_trans

array([[ 4.45136823, -1.00780514, -0.41991729],
       [ 4.12841839, -1.00780514, -0.41991729],
       [ 4.44037315, -0.6918497 , -0.41991729]])

In [27]:
plot_trans - plot_trans.mean(axis=0)

array([[ 0.11131497, -0.10531848,  0.        ],
       [-0.21163487, -0.10531848, -0.        ],
       [ 0.10031989,  0.21063696,  0.        ]])

In [28]:
np.set_printoptions(suppress=True)

In [29]:
plot_trans_ctr = plot_trans - plot_trans.mean(axis=0) - np.array([0,0, 0.1])
plot_trans_ctr

array([[ 0.11131497, -0.10531848, -0.1       ],
       [-0.21163487, -0.10531848, -0.1       ],
       [ 0.10031989,  0.21063696, -0.1       ]])

In [33]:
np.array([51,52,54]).reshape(3,1)

array([[51],
       [52],
       [54]])

In [41]:
plot_trans_csv = np.hstack([np.array([51,52,54]).reshape(3,1), plot_trans_ctr])
plot_trans_csv

array([[51.        ,  0.11131497, -0.10531848, -0.1       ],
       [52.        , -0.21163487, -0.10531848, -0.1       ],
       [54.        ,  0.10031989,  0.21063696, -0.1       ]])

In [42]:
np.savetxt('gcp.csv', plot_trans_csv, delimiter= ',', fmt='%i%f%f%f')

manually remove 51.0 -> 51